In [5]:
!python --version

Python 3.8.8


In [3]:
import random

import pickle

import pandas as pd
import numpy as np
import re
import string
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from dataclasses import dataclass
from typing import List, Callable, Dict, Tuple, Set

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [12]:
def remove_punctuation(text):
    return re.sub('[^a-zA-Z]', '', str(text))

In [9]:
with open('m5_tokenizer.pickle', 'rb') as handle:
    fitted_tokenizer = pickle.load(handle)
    
chosen_model= tf.keras.models.load_model('model5.h5')

In [53]:
def predict_next(seed_text,number_of_words,debug=False):

    next_words = number_of_words
    
    seed_predicted=seed_text
    old_output_word=''

    for _ in range(next_words):
        token_list = fitted_tokenizer.texts_to_sequences([seed_predicted])[0]
        token_list = pad_sequences([token_list], maxlen=19, padding='pre')
        predicted = np.argmax(chosen_model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in fitted_tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
#         if old_output_word==output_word:
#             break
        seed_predicted += " " + output_word
        old_output_word=output_word

    seed_predicted = (seed_predicted + ".").capitalize()
    return(seed_predicted)

In [14]:
easy_q_0=["who","what"]
easy_q_1=["was","is","were","are"]

question_modifiers=["now","then"]

In [7]:
yesno=0 #will indicate whether the bot is expected a yes or no answer

    
noreplies_1=["no","sorta","maybe","kinda"]
noreplies_2=["not really","sort of","kind of","yes but"]

In [59]:
def chat_response(user_text,debug=False):
    chatbot_resp=''
    chatbot_seed=''
    
    ut_m=user_text.lower().split()
    ut_m=[remove_punctuation(x) for x in ut_m]
    
    idontgetit=0
    
    
    if len(ut_m)>6:
        chatbot_resp="SB: Forgive me. Your query was too long. Please use less than 6 terms."
        return chatbot_resp
    
    for term in ut_m:
        if fitted_tokenizer.texts_to_sequences([term])==[[]]:
            chatbot_resp="SB: Forgive me. I am not familiar with the term \""+term+"\". Please reword your query.\n"
            return chatbot_resp
    
    if len(ut_m)==1:
        if ut_m[0]=="suggestion":
            chatbot_resp="SB: Perhaps try asking a query such as \"Who are Romeo and Juliet?\" or \"What is fate?\""
            return chatbot_resp
        else:
            chatbot_resp="SB: I will attempt to complete your query, \""+ut_m[0]+"\"...\n"
            chatbot_seed=ut_m[0]
    
    elif (ut_m[0] in easy_q_0):
        useit=[ut_m[0]]
        if useit!=["who"]:
            useit=["that"]
            
        if (ut_m[1] in easy_q_1):
            if (ut_m[1] in ["is","was"]):
                theone=["the one"]
            else:
                theone=["the ones"]
            
            if (ut_m[2][-3:]=="ing"):
                chatbot_seed=" ".join(theone+useit+ut_m[1:]+["is"])
            else:
                chatbot_seed=' '.join(ut_m[2:]+[ut_m[1]])
                
        elif (ut_m[1][-2:]=="ed"):
            chatbot_seed=' '.join(ut_m[2:]+["was"]+[ut_m[1]]+["by"])
        elif (ut_m[1][-1]=="s"):
            chatbot_seed=" ".join(["the one"]+useit+ut_m[1:]+["is"])
        else:
            idontgetit=1
    elif (ut_m[0]=="where") and (ut_m[1] in easy_q_1):
        chatbot_seed=' '.join(ut_m[2:]+[ut_m[1]]+["in"])
        
    elif ut_m[:3]=="tell me about".split():
        chatbot_seed=' '.join(ut_m[3:])
    else:
        idontgetit=1
    
    if (idontgetit==1) or (chatbot_seed==''):
        chatbot_resp="SB: I do not understand your query, but I shall endeavor to answer it.\n"
        chatbot_seed=' '.join(ut_m)
    
    wordspred=predict_next(chatbot_seed,7)
    
    chatbot_resp=chatbot_resp+"SB: \""+wordspred+"\"\n\n"
    chatbot_resp=chatbot_resp+"SB: Does this answer your query?\n"
    yesno=1
    
    return chatbot_resp

In [48]:
print(chat_response("tell me about juliet",debug=True))

>> Juliet know if i love no true o.

SB: Does this answer your query?



In [50]:
x=[1,2,3]

print(x[1])

2


In [60]:
flag=True

yesno=0

print("SB: Greetings and salutations. I am ShakspeareBot.")
print("SB: I will endeavor to answer your queries.")
print("SB: To end our dialogue, say \"Goodbye\"")
print()
print("SB: What is your query?\n")

while flag==True:
    user_resp=input()
    user_resp=user_resp.lower()
    ur_split=user_resp.split()
    if user_resp=="goodbye" or user_resp[:-1]=="goodbye":
        flag=False
        print("\nSB: Farewell.")
    
    elif ur_split[0] in noreplies_1 or ur_split[:2] in noreplies_2:
        print("\nSB: I am sorry I was unable to answer your query.")
        print("SB: Perhaps try re-wording your query.\n")
        
    elif ur_split[0]=="yes" or ur_split[0]=="yeah":
        print("\nSB: I am pleased I was able to help.\nSB: What else would you like to ask me?\n") 
    else:
        sb_resp=chat_response(user_resp)
        print()
        print(sb_resp)
    

SB: Greetings and salutations. I am ShakspeareBot.
SB: I will endeavor to answer your queries.
SB: To end our dialogue, say "Goodbye"

SB: What is your query?

Who is Juliet Capulet?

SB: "Juliet capulet is to the natural in the most lamentable."

SB: Does this answer your query?

Who is Caesar?

SB: "Caesar is turnd knee many as which can hang."

SB: Does this answer your query?

Who was Caesar?

SB: "Caesar was turnd that in mexico known to make."

SB: Does this answer your query?

Who was Shakespeare?

SB: Forgive me. I am not familiar with the term "shakespeare". Please reword your query.

Where is Rome?

SB: "Rome is in thee at the citadel and i do."

SB: Does this answer your query?

Tell me about Romeo

SB: "Romeo doff for you for ways cassio news."

SB: Does this answer your query?

Goodbye

SB: Farewell.
